In [1]:
import os, sys

if os.path.abspath(".") == "/content":  # google colab
    %pip install sktime keras==2.8.0 tensorflow==2.8.2 keras-nlp==0.4.0 mlflow GitPython tensorflow_addons==0.17.1 databricks-cli  --quiet
    from google.colab import drive

    drive.mount("/content/drive")

    os.chdir("/content/drive/My Drive/master-thesis/src")

In [2]:
from experiments import train_source_model, train_destination_model, train_dest_model_no_weights
from mlflow_logging import MlFlowLogging
import mlflow

In [3]:
mlflow.set_experiment("Transfer learning - same category, other datasets")
mlflow_logging = MlFlowLogging()
run = mlflow.start_run()
mlflow.tensorflow.autolog()
category = "ECG"
dataset = "ECG200"
source_model = train_source_model(category=category, dataset=dataset)
destination = train_destination_model(dataset=dataset, source_model=source_model)
plain_destination = train_dest_model_no_weights(
    model=destination["model"], dataset=dataset
)
history = {**destination["history"], **plain_destination["history"]}
mlflow_logging.log_history(history)


INFO:root:Loading only one category: ECG
INFO:root:Loading only one category: ECG


AttributeError: 'OneHotEncoder' object has no attribute 'categories_'